# Faire des statistiques


In [1]:
import pandas as pd
df = pd.read_csv("../../data/CSS_exact_openalex.csv", low_memory=False)

Deux variables :
- dépendante : nombre de citations `cited_by_count`
- indépendantes : années ; langue ; nombre auteurs, language `language`, open accès `open_access.is_oa`, type `type`

In [57]:
df["authorships.raw_author_name"].isna().sum()

0

Traitement des données

In [58]:
# enlever les lignes sans auteurs
df = df[~df["authorships.raw_author_name"].isna()]

# ajouter le nombre d'auteurs
df["nb_auteurs"] = df["authorships.raw_author_name"].apply(lambda x: len(x.split("|")))

# langue anglais
df = df[df["language"] == "en"]

# renommer les colonnes
df = df.rename(columns={"open_access.is_oa":"openaccess"})

# numérique pour des variables
df["openaccess"] = df["openaccess"].astype(int)

## Univarié

Indicateurs

Distribution

recodage

## Bivarié

Quanti/quanti .corr

In [ ]:
from scipy.stats import pearsonr



PearsonRResult(statistic=0.06316485952917264, pvalue=0.019027924323569402)

Quanti/quali

.groupby

In [ ]:
from scipy.stats import f_oneway

F_onewayResult(statistic=10.73068474537501, pvalue=1.5652446052537652e-08)

Possibilité d'utiliser d'autres bibliothèques, comme pinguin https://pingouin-stats.org/build/html/index.html

In [60]:
#pip install pingouin

,Source,SS,DF,MS,F,p-unc,np2
0,reseaux_n,961.069233,4,240.267308,1.866877,0.11399,0.006842
1,Within,139510.930767,1084,128.700121,NaN,NaN,NaN


Quali/Quali

pd.crosstab

In [39]:
from scipy.stats import chi2_contingency

## Modèles

### Régression linéaire

In [53]:
import statsmodels.api as sm


X = df[['publication_year', 'retracted', 'nb_auteurs']] 
X = sm.add_constant(X) 
y = df["cited_by_count"]

model = sm.OLS(y, X).fit()


print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         cited_by_count   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     31.00
Date:                Mon, 09 Jun 2025   Prob (F-statistic):           7.03e-14
Time:                        20:32:37   Log-Likelihood:                -8549.1
No. Observations:                1312   AIC:                         1.710e+04
Df Residuals:                    1309   BIC:                         1.712e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.501e+04   1991.207  

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


#### Régression logistique

Quels sont les prédicteurs du fait d'avoir plus de 10 citations ?

Création d'une variable 0 (pas du tout ou rarement) et 1 sinon

In [10]:
df["citations10"] = df["cited_by_count"] > 10

Avec Scikit-learn

In [17]:
from sklearn.linear_model import LogisticRegression

In [ ]:
df["open"]

In [25]:
X = pd.get_dummies(df[["publication_year", "openaccess"]], drop_first=True)
y = df["citations10"]

In [18]:
modele = LogisticRegression()
modele.fit(X, y)
modele.coef_

array([[-0.21808669,  1.29588588]])

Avec Statsmodels

In [19]:
import statsmodels.api as sm
from patsy import dmatrices

In [37]:
y,X = dmatrices('citations10 ~ C(openaccess) + publication_year', 
                data = df, 
                return_type = 'dataframe')

In [38]:
modele = sm.Logit(y["citations10[True]"],X)
resultat = modele.fit()
resultat.summary()

Optimization terminated successfully.
         Current function value: 0.451438
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:      citations10[True]   No. Observations:                 1312
Model:                          Logit   Df Residuals:                     1309
Method:                           MLE   Df Model:                            2
Date:                Mon, 09 Jun 2025   Pseudo R-squ.:                  0.1422
Time:                        20:27:30   Log-Likelihood:                -592.29
converged:                       True   LL-Null:                       -690.49
Covariance Type:            nonrobust   LLR p-value:                 2.238e-43
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               441.6868     36.983     11.943      0.000     369.201     514.173
C(openaccess)[T.True]     1.3316      0.166      8.035      0.000       1.007       1.656
publication_year         -0.2199      0.018    -11.981      0.000      -0.256      -0.184
=========================================================================================
"""

Avec la version toute intégrée

In [42]:
#!pip install -U pyshs

In [40]:
import pyshs

In [43]:
pyshs.regression_logistique(df, "citations10", ["openaccess", "publication_year"],)

OR       p            IC 95%
Variable         Modalité                                 
.Intercept                   0.0  0.0***  0.00 [0.00-0.00]
openaccess       False       ref                          
                 True       0.26  0.0***  0.26 [0.19-0.37]
publication_year numérique  1.25  0.0***  1.25 [1.20-1.29]